# LLM

hands-on practice

In [2]:
!python -m pip install -r requirements.txt

In [1]:
# init device
import torch


if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name = 'microsoft/phi-3-mini-4k-instruct'
# trust_remote_code=False 解决：
# 'DynamicCache' object has no attribute 'get_max_length'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device.type,
    torch_dtype='auto',
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
prompt = 'This is a demo of the LLM. Please tell me what is LLM?<|assistant|>'

input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

generation_output = model.generate(
    input_ids=input_ids,
)

print(tokenizer.decode(generation_output[0]))

This is a demo of the LLM. Please tell me what is LLM?<|assistant|> LLM stands for "Large Language Model." It is a type of artificial intelligence (AI)


In [11]:
# check input_ids
print(input_ids)

for input_id in input_ids:
    print(tokenizer.decode(input_id))

tensor([[  910,   338,   263, 13455,   310,   278,   365, 26369, 29889,  3529,
          2649,   592,   825,   338,   365, 26369, 29973, 32001]],
       device='mps:0')
This is a demo of the LLM. Please tell me what is LLM?<|assistant|>


## tokenizer 对比

In [1]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
# test color
for i in range(256):
    print(f'\x1b[0;37;48;5;{i}m Test with {i}\x1b[0m')

In [26]:
colors_list = list(range(100, 107))

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer.encode(sentence)
    for i, t in enumerate(token_ids):
        print(
            f'\x1b[0;37;48;5;{colors_list[i%len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' ',
        )



### BERT base model (insensitive to case; ci)

* 分词法：WordPiece
* 参考文献：Japanese and Korean Voice Search
* 特殊 token
  * 未知：`[UNK]`
  * 分隔符：`[SEP]`
  * 填充：`[PAD]`
  * 分类：`[CLS]`，用于分类任务
  * 掩码：`MASK`，用于隐藏token

特点：

* 不处理换行符，所以无法识别通过换行符体现的信息
* 文本都转为小写
* 没有`##`前缀的`token`表示前面应该加一个空格
* 中文和表情替换为位置 token，即`[UNK]`

In [8]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

### BERT base model (sensitive to case)

* 特殊 token 和ci相同

特点：

* 除了和 ci 相似的特性外
* 有大小写区别
* CAPITALIZATION 分词策略不同


In [ ]:
show_tokens(text, "bert-base-cased")

### GPT-2

* 分词法：BPE
* 参考文献：Neural Machine Translation of Rare Words with Subwords Units
* 特殊token：
  * `<|endoftext|>`

特点：

* 保留换行符
* 保留大小写
* 处理emoji
* 制表符处理

In [9]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

### FLAN-T5

* 分词法：SentencePiece
* 参考文献：SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing
* 特殊 token：
  * 未知：`<unk>`
  * 填充：`<pad>`

特点：

* 没有换行符和空白符
* 不处理 emoji 和中文

In [ ]:
show_tokens(text, "google/flan-t5-small")

### GPT-4

* 分词法：BPE
* 特殊token:
  * `<|endoftext|>`
  * 中间填充token，用于使 llm 能在考虑前后文的情况下生成补全内容：`<|fim_prefix|>`，`<|fim_middle|>`，`<|fim_suffix|>`

特点：

* 各种长度的空白符有特定的token表示
* python 关键字 elif 有特殊的token；对代码的关注
* 更少的token 来表示大多数词：例如 可以比较 CAPITALIZATION 的区别

In [10]:
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

### StarCoder2

专注于代码生成

* 分词法：BPE
* 参考文献：StarCoder2 and The Stack v2: The Next Generation; StarCoder: may the Source be with You!
* 特殊次元：
  * `<|endoftext|>`
  * 中间填充：`<fim_prefix>`，`<fim_middle>`，`<fim_suffix>`，`<fim_pad>`
  * 文件、仓库：`<filename>`，`<reponame>`，`<gh_stars>`

特点：

* 类似GPT-4，各种长度空白符有单独的 token
* 每个数字分配了 token，该设计假设可以更好地表示数字和数学的概念

In [11]:
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/353 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

### Galactica

专注于科学领域

* 分词方法：BPE
* 参考文献：Galactica: A Large Language Model for Science
* 特殊次元：
  * `<s>`
  * `<pad>`
  * `</s>`
  * `<unk>`
  * 引用：`[START_REF]`，`[END_REF]`
  * 逐步推理：`<work>`

特点

* 除了空白符，制表符也做了合并为一个 token 处理

In [12]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

### Phi-3 & Llama 2

Phi-3 重用了llama2

* 分词法: BPE
* 特殊token:
  * `<|endoftext|>`
  * 对话token: `<|user|>`，`<|assistant|>`，`<|system|>`

In [27]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

## embedding 嵌入

* token embedding
* text embedding: 文本嵌入，一段文本输出一个 vector

In [2]:
# 文本嵌入
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
vector = model.encode("Hello world!!!")

print(vector)
print(vector.shape)

[ 3.23563931e-03  3.75038236e-02 -8.97983089e-03 -1.12057838e-03
  8.67718384e-02 -7.09403306e-03  1.40363593e-02  5.02491742e-02
  3.62240971e-04 -1.72925852e-02 -2.42411476e-02 -4.44700010e-02
 -1.85811017e-02  4.49257232e-02  7.61594623e-02 -6.75336272e-02
  6.65571839e-02  6.80227252e-03 -4.35344838e-02  1.25255799e-02
  3.17336209e-02  2.38824263e-02  8.99560656e-03  7.08487481e-02
  2.36419104e-02 -2.50046216e-02  9.15727951e-03  5.60097909e-03
  3.62668075e-02 -7.60861812e-03 -5.13904952e-02  1.26744937e-02
  5.45808189e-02  5.19572832e-02  2.13643852e-06 -4.83830757e-02
  1.63214374e-02  5.49030863e-03 -8.27288069e-03 -3.92285502e-03
  2.43315194e-03  5.29484041e-02 -2.93909255e-02 -4.72563971e-03
  4.67390474e-03 -6.34189770e-02  9.40909144e-03 -2.50146695e-04
  3.11553944e-02  1.51310852e-02 -1.04794349e-03  2.88592521e-02
 -6.74251691e-02  6.58996217e-03  5.90331629e-02  1.05538657e-02
  2.85545811e-02  2.74006631e-02  5.95161552e-03 -2.89275683e-02
  6.13908544e-02 -4.74571

#### 其他词嵌入

In [2]:
import gensim

model = gensim.downloader.load("glove-wiki-gigaword-50")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject